In [211]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import vaex as vx
import sys
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import itertools
from tqdm.contrib.itertools import product
import pymorphy2
%matplotlib inline


tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexkon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
#!pip3 install vaex
#!pip3 install autocorrect
#!pip3 install fuzzywuzzy
#!pip3 install python-Levenshtein
#!pip3 install pymorphy2

In [ ]:
%%time
data = vx.open('history_small.csv')
data.__delitem__('weekday')
data.__delitem__('time')
data.__delitem__('Unnamed: 0')
data = data[['UQ', 'cnt']]

In [55]:
morph = pymorphy2.MorphAnalyzer()

In [5]:
data = data.dropna(column_names=['UQ'])
data['UQ'] = data['UQ'].str.lower()

In [8]:
source = list(data['UQ'].unique())

In [9]:
stop_words = stopwords.words('russian')

In [11]:
%%time
key_source = data['UQ'].values

CPU times: user 12.6 s, sys: 9.82 s, total: 22.4 s
Wall time: 45.7 s


In [158]:
word = {}
for i in tqdm(range(len(source))):
    string = re.sub("[^0-9a-zа-я]+", " ", source[i])
    x = string.split(' ')
    for j in x:
        if j in word.keys():
            word[j] +=1
        else:
            word[j] = 1

100%|██████████| 15148855/15148855 [00:55<00:00, 271875.60it/s]


In [159]:
words = pd.DataFrame.from_dict(word, orient='index').reset_index()
words.rename({0:'num'}, axis=1, inplace=True)
print(words.shape)
words = words[words['index'].str.contains(r'[a-zа-я]')]
print(words.shape)
words.sort_values(by='num', ascending=False, inplace=True)
words_nostop = words[~words['index'].isin(stop_words)]
top_words = words_nostop[words_nostop.num>=5]

(2734432, 2)
(2551276, 2)


In [173]:
POS_list = ['ADJF']
wd_list = list(words_nostop['index'])
norm_words = {}
for i in tqdm(range(len(wd_list))):
    p = morph.parse(wd_list[i])
    s = set(map(lambda l: l.normal_form, p))
    st = set(map(lambda l: l.tag.POS, p))
    if len(s) == 1:
        norm_words[wd_list[i]] = p[0].normal_form
    elif st == {'ADJF'}:
        norm_words[wd_list[i]] = p[0].normal_form
    else:
        norm_words[wd_list[i]] = wd_list[i]

100%|██████████| 2551125/2551125 [06:47<00:00, 6254.06it/s]


In [203]:
words_clear = pd.DataFrame.from_dict(norm_words, orient='index', columns = ['word']).reset_index()
words_clear = words_clear.merge(words_nostop, how='left', on='index')
words_clear = words_clear.groupby('word').sum().reset_index().sort_values(by='num', ascending=False)
words_clear.rename({'word': 'index'}, axis=1, inplace=True)
words_clear = words_clear.loc[words_clear.num>=words_clear.num.quantile(0.95)].copy()

In [ ]:
# spellru = Speller(lang='ru')
# spellen = Speller(lang='en')

# patternen = re.compile("[a-z]+")
# patternru = re.compile("[а-я]+")

# misswords = list(word.keys())
# correction = {}
# for x in tqdm(range(len(misswords))):
#     if len(misswords[x])<3:
#         correction[misswords[x]] = misswords[x]
#         continue
#     if patternen.fullmatch(misswords[x]):
#         correction[misswords[x]] = spellen(misswords[x])
#     elif patternru.fullmatch(misswords[x]):
#         correction[misswords[x]] = spellru(misswords[x])
#     else:
#         correction[misswords[x]] = misswords[x]

In [27]:
# spellru = SpellChecker(language='ru')
# misswords = list(word.keys())
# correction = {}
# misspelled = spellru.unknown(misswords)

In [ ]:
# new_doc = TextBlob(' '.join(list(misspelled)))
# result = new_doc.correct()

In [ ]:
# word_dict = {}
# word_list = list(word.keys())
# tops = list(top_words['index'])
# for j in tqdm(range(len(word_list))):
#     word_input = word_list[j]
#     temp = {}
#     for i in range(len(tops)):
#         rat = fuzz.ratio(word_input, tops[i])
#         if rat>50:
#             temp[tops[i]] = rat
#     compare = pd.DataFrame.from_dict(temp, orient='index').reset_index()
#     compare.rename({0: 'simi'}, axis=1, inplace=True)
#     compare = compare.merge(top_words, how='left', on='index')
#     compare['metric'] = (0.01*compare['simi'])*np.log(compare['num'])
#     compare.sort_values(by='metric', ascending=False, inplace=True)
#     key_word = compare['index'].iloc[0]
#     word_dict[word_input] = key_word

In [244]:
double_words = {}
triple_words = {}
for i in tqdm(range(len(source))):
    x = key_source[i].as_py().split(' ')
    s = ''
    if (len(x))>=2:
        if not any(list(map(lambda w: w in stop_words,x[:2]))):
            s = ' '.join(x[:2])
            if s in double_words.keys():
                double_words[s] += 1
            else:
                double_words[s] = 1
        s= ''
        if len(x)>2:
            s = ' '.join(x[:3])
            if s in triple_words.keys():
                triple_words[s] += 1
            else:
                triple_words[s] = 1

100%|██████████| 15148855/15148855 [01:38<00:00, 154013.78it/s]


In [245]:
double_words = pd.DataFrame.from_dict(double_words, orient='index').reset_index()
double_words.rename({0:'num'}, axis=1, inplace=True)
double_words.sort_values(by='num', ascending=False, inplace=True)
double_words = double_words[double_words['num']>= 50]

triple_words = pd.DataFrame.from_dict(triple_words, orient='index').reset_index()
triple_words.rename({0:'num'}, axis=1, inplace=True)
triple_words.sort_values(by='num', ascending=False, inplace=True)
triple_words = triple_words[triple_words['num']>= 50]

In [404]:
zz = double_words.copy()
zz['index_modi'] = zz['index'].str.split(' ').apply(lambda x: ' '.join(sorted(x)))
temp = zz.groupby('index_modi').sum().reset_index()
zz.drop_duplicates(subset=['index_modi'], inplace=True)
zz = temp.merge(zz[['index', 'index_modi']], how='left', on='index_modi')
double_words = zz[['index', 'num']].sort_values(by='num')


zz = triple_words.copy()
zz['index_modi'] = zz['index'].str.split(' ').apply(lambda x: ' '.join(sorted(x)))
temp = zz.groupby('index_modi').sum().reset_index()
zz.drop_duplicates(subset=['index_modi'], inplace=True)
zz = temp.merge(zz[['index', 'index_modi']], how='left', on='index_modi')
triple_words = zz[['index', 'num']].sort_values(by='num')

del zz

In [458]:
words_clear['source'] = 'words_clear'
double_words['source'] = 'double_words'
triple_words['source'] = 'triple_words'
words_nostop['source'] = 'words_nostop'
total_df = pd.concat([words_clear, double_words, triple_words, words_nostop], ignore_index=True)
total_df.to_csv('data_source.csv')
dd = pd.DataFrame(source)
dd.to_csv('source_req.csv')

<ipython-input-458-1021644846b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  words_clear['source'] = 'words_clear'
<ipython-input-458-1021644846b1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  words_nostop['source'] = 'words_nostop'


### Расчет для тегов

In [355]:
def stops(x):
    x = x.split(' ')
    if x[-1] in stop_words:
        return 1
    return 0

In [443]:
word_phrase = 'резиновый'
word_phrase = word_phrase.split(' ')
sign = 0
word_list = list(words_clear['index'])
key_words = []
for word_input in word_phrase:
    if word_input in stop_words:
        if word_input == 'без':
            sign='-'
            continue
        else:
            sign='+'
            continue
    temp = {}
    for i in tqdm(range(len(word_list))):
        rat = fuzz.ratio(word_input, word_list[i])
        if rat>50:
            temp[word_list[i]] = rat
    compare = pd.DataFrame.from_dict(temp, orient='index').reset_index()
    compare.rename({0: 'simi'}, axis=1, inplace=True)
    compare = compare.merge(words_clear, how='left', on='index')
    compare['metric'] = (0.01*compare['simi'])*np.log(compare['num'])
    compare.sort_values(by='metric', ascending=False, inplace=True)
    key_words.append(compare['index'].iloc[0])

100%|██████████| 123430/123430 [00:00<00:00, 379903.84it/s]


In [444]:
using = {}
top_head_dict = {1:10, 2:6, 3:3, 4:3}
for key_word in key_words:   
    temp = {}
    for i in tqdm(range(len(source))):
        if key_word in source[i]:
            x = source[i].split(' ')
            for j in x:
                if j in temp.keys():
                    temp[j] +=1
                else:
                    temp[j] = 1
    most_used = pd.DataFrame.from_dict(temp, orient='index').reset_index()
    most_used.rename({0:'use_num'}, axis=1, inplace=True)
    most_used.sort_values(by='use_num', ascending=False, inplace=True)
    most_used = most_used[~most_used['index'].isin(stop_words)]
    most_used = most_used.head(top_head_dict[len(key_words)])
    most_used = most_used.merge(words_nostop, how='left', on='index')
    using[key_word] = most_used

100%|██████████| 15148855/15148855 [00:06<00:00, 2172460.33it/s]


In [445]:
key_list = []
weight_list = []
matching_list = {}
metric_list = {}
match_size = 1
for key_word in using.keys():  
    key_list.append(list(using[key_word]['index']))
    weight_list.append(list(using[key_word]['use_num']))
    match_size *= len(using[key_word])
weights = list(product(*weight_list))
max_weight = np.max(weights)
for i in range(len(weights)):
    weights[i] = np.sum(list(weights[i]))
step = 0
for elem in product(*key_list):
    cur_weight = 100*weights[step]/max_weight
    step+=1
    for row in double_words.itertuples():
        t = fuzz.token_sort_ratio(row[1], str(' '.join(elem)))
        if t>50:
            metr = t*cur_weight*np.log(np.log(row[2]))
            if row[1] in matching_list.keys():
                if metr>metric_list[row[1]]:
                    metric_list[row[1]] = metr
            else:
                matching_list[row[1]] = 1
                metric_list[row[1]] = metr
    for row in triple_words.itertuples():
        t = fuzz.token_sort_ratio(row[1], str(' '.join(elem)))
        if t>50:
            metr = t*cur_weight*np.log(np.log(row[2]))
            if row[1] in matching_list.keys():
                if metr>metric_list[row[1]]:
                    metric_list[row[1]] = metr
            else:
                matching_list[row[1]] = 1
                metric_list[row[1]] = metr
for x in matching_list.keys():
    matching_list[x] = matching_list[x]*metric_list[x]
matches = pd.DataFrame.from_dict(matching_list, orient='index').reset_index()
matches.rename({0:'num'}, axis=1, inplace=True)
matches.sort_values(by='num',  ascending=False, inplace=True)
matches = matches.head(50)
matches['flag'] = matches['index'].apply(stops)
matches = matches.loc[matches.flag == 0].copy()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [451]:
print('Запрос:')
print(' '.join(word_phrase))
print('\nТеги:')
list(matches.head(10)['index'])

Запрос:
резиновый

Теги:


['резиновые сапоги',
 'резиновый коврик',
 'мяч резиновый',
 'новый год',
 'резиновый член',
 'резиновые тапочки',
 'протеиновый батончик',
 'перчатки резиновые',
 'резиновые ботинки',
 'тапки резиновые']